<a href="https://colab.research.google.com/github/carlos-souza-blip/Trabalho-de-Analise-de-Risco/blob/main/Test_RHMP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# iNCLUINDO O BANCO DE DADOS
from google.colab import files
uploaded = files.upload()

In [ ]:
# INCLUINDO LIVRARIAS
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, roc_curve, classification_report,\
                            accuracy_score, confusion_matrix, auc
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [ ]:
# ABRINDO O BANCO DE DADOS
dados = pd.read_excel('rmhp_selecao_go.xlsx')
dados

,cnes,sexo,idade,diag_princ,dias_perm,ano,fxidade,cid50,permanencia,uso_uti,obito,icc,ieh
0,2333988,1,44,A09,3,2015,1,0,2,0,0,0,0
1,2333988,1,57,A491,2,2015,2,1,2,0,0,0,0
2,2333988,3,18,B188,4,2015,1,0,2,0,0,1,0
3,2333988,1,49,I10,2,2015,1,0,2,0,0,0,1
4,2333988,3,44,J068,2,2015,1,0,2,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1004665,9813349,3,44,S520,1,2021,1,0,1,0,0,0,0
1004666,9813349,3,50,S521,2,2021,2,0,2,0,0,0,0
1004667,9813349,3,64,S522,1,2021,3,0,1,0,0,0,0
1004668,9813349,3,97,M169,4,2021,6,0,2,0,0,0,0


In [ ]:
# ORGANIZANDO AS VARIAVEIS PARA O TESTE
dados = dados[['ano', 'obito', 'icc', 'ieh', 'cid50', 'uso_uti', 'permanencia', 'fxidade']]
dados

,ano,obito,icc,ieh,cid50,uso_uti,permanencia,fxidade
0,2015,0,0,0,0,0,2,1
1,2015,0,0,0,1,0,2,2
2,2015,0,1,0,0,0,2,1
3,2015,0,0,1,0,0,2,1
4,2015,0,0,0,0,0,2,1
...,...,...,...,...,...,...,...,...
1004665,2021,0,0,0,0,0,1,1
1004666,2021,0,0,0,0,0,2,2
1004667,2021,0,0,0,0,0,1,3
1004668,2021,0,0,0,0,0,2,6


In [ ]:
# APLICANDO O MODELO BINOMIAL PARA VERIFICAR A ESTRUTURA DOS DADOS
modelo = smf.glm(formula= 'obito ~ icc + ieh + uso_uti + permanencia + cid50 + fxidade', data=dados,
                 family = sm.families.Binomial()).fit()
print(modelo.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  obito   No. Observations:              1004670
Model:                            GLM   Df Residuals:                  1004663
Model Family:                Binomial   Df Model:                            6
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -1.7905e+05
Date:                Tue, 29 Jun 2021   Deviance:                   3.5809e+05
Time:                        14:30:17   Pearson chi2:                 8.45e+05
No. Iterations:                     7                                         
Covariance Type:            nonrobust                                         
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      -4.7516      0.014   -340.707      

In [ ]:
# APLICANDO OUTRO TESTE PARA VERIFICAR A ESTRUTURA DO MODELO
model = LogisticRegression(penalty='none', solver='newton-cg')
y = dados.obito
x = pd.get_dummies(dados[['icc', 'ieh', 'uso_uti', 'permanencia', 'fxidade', 'cid50']])

In [ ]:
predictions = model.predict(x)
print ('GFT + Wiki / GT R-squared: %.4f' % model.score(x, y))

GFT + Wiki / GT R-squared: 0.9383


In [ ]:
prediction_error = y - predictions
print(prediction_error)

0          0
1          0
2          0
3          0
4          0
          ..
1004665    0
1004666    0
1004667    0
1004668    0
1004669    0
Name: obito, Length: 1004670, dtype: int64


In [ ]:
# é necessário adicionar uma constante a matriz X
x = sm.add_constant(x)
# OLS vem de Ordinary Least Squares e o método fit irá treinar o modelo
results = sm.OLS(y, x).fit()
# mostrando as estatísticas do modelo
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  obito   R-squared:                       0.145
Model:                            OLS   Adj. R-squared:                  0.145
Method:                 Least Squares   F-statistic:                 2.838e+04
Date:                Tue, 29 Jun 2021   Prob (F-statistic):               0.00
Time:                        14:41:13   Log-Likelihood:                 94985.
No. Observations:             1004670   AIC:                        -1.900e+05
Df Residuals:                 1004663   BIC:                        -1.899e+05
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const          -0.0230      0.001    -38.791      

In [ ]:
print ("GFT + Wiki / GT  R-squared", results.rsquared)

GFT + Wiki / GT  R-squared 0.14492705090180613


In [ ]:
# mostrando as previsões para o mesmo conjunto passado
print(modelo.predict(x))

0          0.012972
1          0.035339
2          0.017581
3          0.014752
4          0.012972
             ...   
1004665    0.012816
1004666    0.019139
1004667    0.027823
1004668    0.086608
1004669    0.105900
Length: 1004670, dtype: float64


In [ ]:
# Visualizar os coeficientes encontrados
model.fit(x, y)
print(model.coef_)

[[-2.37578612  0.30870939  0.1303971   2.29891577  0.01221132  0.3952291
   0.62989031]]


In [ ]:
# Aplicando o Teste T-STUDENT
print(np.exp(results.params[1:]))

icc            1.018516
ieh            0.998808
uso_uti        1.286353
permanencia    0.999794
fxidade        1.020273
cid50          1.041412
dtype: float64


In [ ]:
# Aplicando Binomial
print(np.exp(modelo.params[1:]))

icc            1.361667
ieh            1.139281
uso_uti        9.963374
permanencia    1.012286
cid50          1.877405
fxidade        1.484724
dtype: float64


In [ ]:
# Visualizando o R^2
model.score(x, y)

0.9382732638577842

In [ ]:
# Mostrando as previsões 
model.predict_proba(x)

array([[0.9870282 , 0.0129718 ],
       [0.96466139, 0.03533861],
       [0.98241921, 0.01758079],
       ...,
       [0.97217697, 0.02782303],
       [0.91339216, 0.08660784],
       [0.89410026, 0.10589974]])